# PROYECTO FINAL

## 1. MODELO LLM

In [1]:
from openai import OpenAI
import json
import getpass

api_key = getpass.getpass("Enter your OpenAI API Key:")

client_openai = OpenAI(api_key = api_key)

Enter your OpenAI API Key:··········


### 1.1. Input del usuario

In [2]:
def user_input():
    question = input("Escriba su consulta:")
    return question

### 1.2. Prompt Engineering con ChatGPT

In [23]:
def input_format(question, model="gpt-3.5-turbo", temperature=0):

    prompt_template = """
Eres un experto de SQL para una base de datos de registro de vehiculos. Tu tarea es convertir una pregunta de lenguaje natural a una consulta SQL valida.

## **ESTRUCTURA DE LA BASE DE DATOS**
Nombre del proyecto: 'aiparking-451016'
Nombre de la tabla: '00_aiparking.vehiculos'

- matricula_id STRING
- marca_ds STRING
- modelo_ds STRING
- color_ds STRING
- tipo_vehiculo_id STRING

## **TAREA**
Responde UNICAMENTE con la consulta en SQL.

## **EJEMPLOS**
Pregunta: "Cual es el modelo del vehiculo con la matricula 5489 HEB?"
Respuesta: SELECT modelo_ds FROM `aiparking-451016.00_aiparking.vehiculos` WHERE matricula_id = '5489 HEB';

Pregunta: "Que tipo de vehiculo es el Ford Focus?"
Respuesta: SELECT tipo_vehiculo_id FROM `aiparking-451016.00_aiparking.vehiculos` WHERE marca_ds = 'Ford' AND modelo_ds = 'Focus';

## **NUEVA PREGUNTA DEL USUARIO:**
"{question}"

## **RESPUESTA EN SQL:**
"""
    prompt = prompt_template.format(question=question)
    messages = [{"role": "user", "content": prompt}]
    response = client_openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

    sql_query = response.choices[0].message.content
    return sql_query


* Ejemplo 1

In [4]:
question_1 = "Cuál es el color del coche con matricula 1456LED?"

In [5]:
response_1 = input_format(question_1)
print(response_1)

SELECT color_ds FROM `aiparking-451016.00_aiparking.vehiculos` WHERE matricula_id = '1456LED';


### 1.3. Acceso a la Database de BigQuery

In [6]:
from google.colab import auth
auth.authenticate_user()

In [7]:
from google.cloud import bigquery

# Creamos un cliente de BigQuery
client = bigquery.Client(project='aiparking-451016')

In [8]:
import pandas as pd

def sql_ans(query):

      # Consulta SQL
      query_job = client.query(query)

      results = query_job.result()

      # Convertir los resultados a un DataFrame de Pandas
      df = results.to_dataframe()

      return df

### 1.4. Pipeline

In [32]:
def main(question=None):

    # 1. Pregunta del usuario
    if question is None:
      question = user_input()

    # 2. Conversión lenguaje natural a SQL
    query = input_format(question=question, model="gpt-3.5-turbo", temperature=0)

    # 3. Acceso al Database de BigQuery
    answer = sql_ans(query)

    # 6. Mostrar la salida
    print(answer)

    return answer

if __name__ == "__main__":
    main()

Escriba su consulta:coches de color Gris, Blanco o Azul y que sean de tipo 1
  matricula_id marca_ds modelo_ds color_ds tipo_vehiculo_id
0      1234MNO                      Blanco                1
1      0225KKL    Honda     Civic     Azul                1
